In [ ]:
%load_ext autoreload
%autoreload 2
import VAE
from keras.datasets import mnist
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np

## Load MNIST Data and split into test/train

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train_re = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])
x_test_re = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])
print x_train_re.shape
print x_test_re.shape

## Instaniate Poisson-Gaussian VAE and Load Params

In [ ]:
v = VAE.VAE(param_file='../params/poisson_gauss_mnist.json')
print v.params
v.construct()

## Fit the VAE

In [ ]:
v.fit(x_train_re > 0.5, shuffle=True)

In [ ]:
v.model.save_weights('../saved_models/poisson_gauss_mnist.h5')

## Run model on test data

In [ ]:
pred = v.model.predict(x_test_re > 0.5, batch_size=100).reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2])
print pred.shape

In [ ]:
%matplotlib notebook
sample = 300
plt.subplot(1,2,1)
plt.imshow(x_test[sample,:,:])
plt.subplot(1,2,2)
plt.imshow(pred[sample,:,:])

## Encode some test data

In [ ]:
z = v.encode(x_test_re > 0.5, 'mu')
print z.shape

In [ ]:
%matplotlib notebook
plt.scatter(z[:,0],z[:,1], c=y_test)
Z = np.array([[-2,-1],#1
              [3.5,-1],#0
              [1.5,3],#7
              [-0.5,-4]])#6
#plt.scatter(Z[:,0],Z[:,1], c='r')
plt.show()
plt.xlabel('mu_1')
plt.ylabel('mu_2')
#plt.colorbar()

## VAE Decoding

In [ ]:
LOAD_MODEL = True
batch_size = 4
if LOAD_MODEL :
    v.model.load_weights('/home/gridsan/CH24434/school/vae/saved_models/model1_weights.h5')
Z = np.random.normal(size=(batch_size, 2))
Z = np.array([[-2,-1],#1
              [3.5,-1],#0
              [1.5,3],#7
              [-0.5,-4]])#6
x = v.decode(Z, 'Decode1').reshape(batch_size, 28, 28)
print x.shape

In [ ]:
%matplotlib notebook
THRESH = None

for i in np.arange(0,x.shape[0]) :
    plt.subplot(2,2,i+1)
    if THRESH :
        plt.imshow(x[i,:,:] > THRESH)
    else :
        plt.imshow(x[i,:,:])
    #plt.colorbar()
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.gca().axes.get_yaxis().set_visible(False)
plt.show()